## Imports

In [43]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [44]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [45]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [46]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [47]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [48]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [49]:
df['loc']

1     (18555 Collins Ave Apt 4405, Sunny Isles Beach...
2     (50 South Pointe Dr APT 2001, Miami Beach, FL ...
3     (9705 Collins Ave #2004n, Bal Harbour, FL 3315...
4     (1000 South Pointe Dr APT 3502, Miami Beach, F...
5     (5000 Island Estates Dr #1402, Aventura, FL 33...
6     (450 Alton Rd APT 3303, Miami Beach, FL 33139,...
7     (2627 S Bayshore Dr APT 801, Miami, FL 33133, ...
8     (8701 Collins Ave #304, Miami Beach, FL 33154,...
9     (100 Jefferson Ave, Miami Beach, FL 33139, USA...
10    (6000 Island Blvd #2603, Aventura, FL 33160, U...
Name: loc, dtype: object

In [50]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [51]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Porsche Design Tower\r18555 Collins Ave 4405\r...,2/8/2022,6/30/2022,79,"$14,400,000.00",9474,"$2,154.40",Nelson Gonzalez PA,BHHS EWM Realty,Melissa Barragan,Dezer Platinum Realty,Porsche Design Tower 18555 Collins Ave 4405 Su...,Porsche Design Tower,18555 Collins Ave 4405 Sunny Isles Beach,"(18555 Collins Ave Apt 4405, Sunny Isles Beach...","(25.9481501, -80.11994659999999, 0.0)",25.948150,-80.119947,0.0
2,Continuum on South Beach\r50 S Pointe Dr 2001\...,1/17/2022,6/29/2022,70,"$6,500,000.00",1900,"$3,421.05",Edward Cimino,London Foster Realty,Micky Ross-Granot,Palm Props of S Florida Inc,Continuum on South Beach 50 S Pointe Dr 2001 M...,Continuum on South Beach,50 S Pointe Dr 2001 Miami Beach,"(50 South Pointe Dr APT 2001, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0
3,St Regis Bal Harbour\r9705 Collins Ave 2004N\r...,3/17/2022,6/30/2022,61,"$6,200,000.00",3003,"$2,064.60",Ludmila Bogatov,Bogatov Realty,Benjamin Moss,"Compass Florida, Inc",St Regis Bal Harbour 9705 Collins Ave 2004N Ba...,St Regis Bal Harbour,9705 Collins Ave 2004N Bal Harbour,"(9705 Collins Ave #2004n, Bal Harbour, FL 3315...","(25.8892229, -80.1228812, 0.0)",25.889223,-80.122881,0.0
4,Murano at Portofino Condo\r1000 S Pointe Dr 35...,3/30/2022,6/27/2022,69,"$5,125,000.00",2618,"$1,957.60",Dina Goldentayer,Douglas Elliman,Julian Cohen,Douglas Elliman,Murano at Portofino Condo 1000 S Pointe Dr 350...,Murano at Portofino Condo,1000 S Pointe Dr 3502 Miami Beach,"(1000 South Pointe Dr APT 3502, Miami Beach, F...","(25.7690044, -80.1381107, 0.0)",25.769004,-80.138111,0.0
5,Prive Condo\r5000 Island Estates Dr 1402\rAven...,4/7/2022,6/30/2022,25,"$3,325,000.00",2956,"$1,124.83",Monica Sarmiento,One Sotheby's International,Alicia Cuba,"Compass Florida, Inc",Prive Condo 5000 Island Estates Dr 1402 Aventura,Prive Condo,5000 Island Estates Dr 1402 Aventura,"(5000 Island Estates Dr #1402, Aventura, FL 33...","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0
6,Icon South Beach\r450 Alton Rd 3303\rMiami Beach,5/5/2022,6/28/2022,13,"$3,300,000.00",2158,"$1,529.19",Joyce Gato,Douglas Elliman,Christian Kawas,Douglas Elliman,Icon South Beach 450 Alton Rd 3303 Miami Beach,Icon South Beach,450 Alton Rd 3303 Miami Beach,"(450 Alton Rd APT 3303, Miami Beach, FL 33139,...","(25.7719135, -80.1380353, 0.0)",25.771914,-80.138035,0.0
7,The Grovenor House\r2627 S Bayshore Dr 801\rMiami,5/26/2022,6/29/2022,3,"$3,100,000.00",2333,"$1,328.76",Bea Evelyn Citron,"Compass Florida, Inc",Ana Sanchez,One Sotheby's International,The Grovenor House 2627 S Bayshore Dr 801 Miami,The Grovenor House,2627 S Bayshore Dr 801 Miami,"(2627 S Bayshore Dr APT 801, Miami, FL 33133, ...","(25.7324342, -80.2350084, 0.0)",25.732434,-80.235008,0.0
8,Eighty Seven Park\r8701 Collins Ave 304\rMiami...,12/28/2020,6/27/2022,508,"$3,000,000.00",2172,"$1,882.06",Diana Garchitorena,Douglas Elliman,Alexi Altuve,The Keyes Company,Eighty Seven Park 8701 Collins Ave 304 Miami B...,Eighty Seven Park,8701 Collins Ave 304 Miami Beach,"(8701 Collins Ave #304, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
9,The Courts at South Beach\r100 Jefferson Ave 1...,3/24/2022,6/30/2022,47,"$2,900,000.00",2620,"$1,106.87",Vanessa Frank PA,One Sotheby's International,David Sherman,"Compass Florida, Inc",The Courts at South Beach 100 Jefferson Ave 10...,The Courts at South Beach,100 Jefferson Ave 10021 Miami Beach,"(100 Jefferson Ave, Miami Beach, FL 33139, USA...","(25.770563, -80.13698, 0.0)",25.770563,-80.136980,0.0
10,B0ella Mare\r6000 Island Blvd 2603\rAventura,4/19/2022,6/29/2022,1,"$2,900,000.00",4070,$822.00,Richard Goihman,Douglas Elliman,Ronit Perez,Suma Luxury,B0ella Mare 6000 Island Blvd 2603 Aventura,B,0ella Mare 6000 Island Blvd 2603 Aventura,

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [52]:
df.at[10,'building_name']=('Bella Mare')
df.at[10,'address']=('6000 Island Blvd 2603 Aventura')

In [53]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 26th - July 2nd')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [54]:
m.save('index.html')

## Generate Map URL

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [55]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [56]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name  + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_07_02_22
